In [1]:
import pandas as pd
import glob
import json
from IPython.display import clear_output

In [ ]:
all_rows = []
files = glob.glob('./us/*')
for i in range(len(files)):
    clear_output(wait=True)
    print(f"Progress: {i}/{len(files)}")
    with open(files[i]) as f:
        row = [json.loads(l)['properties'] for l in f.readlines()]
        all_rows.extend(row)
clear_output(wait=True)
print(f"Progress: {len(files)}/{len(files)}")

Progress: 20/186


In [ ]:
df = pd.DataFrame(all_rows)

In [ ]:
df.head()